In this assignment we need to slide the point which is in the center of the image to any other
point on that image. The other points on that image will move accordingly. For example in the
following figure:
When we transform the triangle in which there is an arrow, the triangle will get compressed. The
other points in that triangle will change accordingly. Similarly the other triangles will change.
For this assignment one way to do is to find the affine transformation matrix of each source
triangle to the destination triangle. Get the bounding box of the triangle and apply warp affine on
that image included in the bounding box for source triangle to the destination triangle. Then crop
the image outside the triangle destination triangle.
You will learn the following functions in opencv.
getAffineTransform
warpAffine
fillConvexPoly


In [2]:
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In [21]:

# Warps and alpha blends triangular regions from img1 and img2 to img
def warpTriangle(img1, img2, tri1, tri2) :
    
    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(tri1)
    r2 = cv2.boundingRect(tri2)
    
    # Offset points by left top corner of the respective rectangles
    tri1Cropped = []
    tri2Cropped = []
    
    for i in range(0, 3):
        tri1Cropped.append(((tri1[0][i][0] - r1[0]),(tri1[0][i][1] - r1[1])))
        tri2Cropped.append(((tri2[0][i][0] - r2[0]),(tri2[0][i][1] - r2[1])))

    # Crop input image
    img1Cropped = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]

    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(tri1Cropped), np.float32(tri2Cropped) )
    
    # Apply the Affine Transform just found to the src image
    img2Cropped = cv2.warpAffine( img1Cropped, warpMat, (r2[2], r2[3]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    # Get mask by filling triangle
    mask = np.zeros((r2[3], r2[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tri2Cropped), (1.0, 1.0, 1.0), 16, 0);

    img2Cropped = img2Cropped * mask
    
    # Copy triangular region of the rectangular patch to the output image
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] * ( (1.0, 1.0, 1.0) - mask )
    
    img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] = img2[r2[1]:r2[1]+r2[3], r2[0]:r2[0]+r2[2]] + img2Cropped


if __name__ == '__main__' :

    # Read input image
    imgIn = cv2.imread("NOBITA.jpg")

    # Output image is set to white
    imgOut = 255 * np.ones(imgIn.shape, dtype = imgIn.dtype)
    
    # Input triangle
    tri1 = np.float32([[[0,0], [510,0], [250,250]]])
    # Output triangle
    tri2 = np.float32([[[0,0], [510,0], [200,200]]])
    
    # Input triangle
    tri3 = np.float32([[[0,0], [0,510], [250,250]]])
    # Output triangle
    tri4 = np.float32([[[0,0], [0,510], [200,200]]])
    
    # Input triangle
    tri5 = np.float32([[[0,510], [510,510], [250,250]]])
    # Output triangle
    tri6 = np.float32([[[0,510], [510,510], [200,200]]])
    
    # Input triangle
    tri7 = np.float32([[[510,510], [510,0], [250,250]]])
    # Output triangle
    tri8 = np.float32([[[510,510], [510,0], [200,200]]])
    
    
    # Warp all pixels inside input triangle to output triangle
    warpTriangle(imgIn, imgOut, tri1, tri2)
    warpTriangle(imgIn, imgOut, tri3, tri4)
    warpTriangle(imgIn, imgOut, tri5, tri6)
    warpTriangle(imgIn, imgOut, tri7, tri8)    
    
    
    # Draw triangle using this color
    color = (255, 150, 0)

    
    # Draw triangles in input and output images.
    cv2.polylines(imgIn, tri1.astype(int), True, color, 2, 16)
    cv2.polylines(imgOut, tri2.astype(int), True, color, 2, 16)
    
    cv2.polylines(imgIn, tri3.astype(int), True, color, 2, 16)
    cv2.polylines(imgOut, tri4.astype(int), True, color, 2, 16)
    
    cv2.polylines(imgIn, tri5.astype(int), True, color, 2, 16)
    cv2.polylines(imgOut, tri6.astype(int), True, color, 2, 16)  
    
    cv2.polylines(imgIn, tri7.astype(int), True, color, 2, 16)
    cv2.polylines(imgOut, tri8.astype(int), True, color, 2, 16)    
    
    cv2.imshow("Input", imgIn)
    cv2.imshow("Output", imgOut)
    
    cv2.imwrite("Input.jpg", imgIn)
    cv2.imwrite("Output.jpg", imgOut)
    
    cv2.waitKey(0)        